In [38]:
import urllib
r = urllib.urlopen('http://belimukena.com/').read()
from bs4 import BeautifulSoup 
soup = BeautifulSoup(r,"lxml")
links = soup.find_all('a')
all_cat_link = [link for link in links if '/category/' in str(link) and 'javascript:void(0)' not in str(link) and '#' not in str(link)]

In [39]:
category_link = [l["href"] for l in all_cat_link if len(l["href"].split("/"))==6]

In [40]:
from PIL import Image
from StringIO import StringIO
import string
import pymysql

database='etd'
connection=pymysql.connect(cursorclass=pymysql.cursors.DictCursor,
                               db=database,
                               host='localhost',
                               passwd='',
                               port=3306,
                               user='root')
cursor = connection.cursor()

def getProduk(link):
    p = urllib.urlopen(link).read()
    soup_p = BeautifulSoup(p,"lxml")

    nama = soup_p.find("div",{id:"minimarket-right"}).find("h1").text
    kategori = soup_p.find("div","detail").find_all("td")[1].text
    harga = soup_p.find("div","detail").find_all("td")[7].text.replace("Rp","").replace(".","").strip()
    berat = soup_p.find("div","detail").find_all("td")[5].text
    bahan =  soup_p.find("div","post").find("p").text
    gambar = soup_p.find("div","bigpic").find("a")["href"]
    
    rgam = urllib.urlopen(gambar).read()
    i = Image.open(StringIO(rgam))
    
    exclude = set(string.punctuation)
    judul = ''.join(ch for ch in nama if ch not in exclude)
    
    nama_file = "beli_mukena/" + judul + ".jpg"
    i.save(nama_file,'JPEG')
    
    sql = "INSERT IGNORE INTO produk VALUES (%s,%s,%s,%s,%s,%s,%s)"
    cursor.execute(sql,("",str(nama),int(harga),str(berat),str(kategori),str(bahan),nama_file))
    connection.commit()



In [ ]:
produk_link = set()
for l in category_link:
    print(l)
    rc = urllib.urlopen(l).read()
    soup_c = BeautifulSoup(rc,"lxml")
    max_page = int(soup_c.find("span","pages").text.split()[3].replace(":",""))
    for i in range(1,max_page+1):
        page_url = str(l)+"page/"+str(i)
        print(page_url)
        rcc = urllib.urlopen(page_url).read()
        soup_cc = BeautifulSoup(rcc,"lxml")
        links = soup_cc.find_all('a')
        for link in links:
            if len(link["href"].split("/"))==5:
                if link["href"] not in produk_link:
                    try:
                        getProduk(link["href"])
                        produk_link.add(link["href"])
                    except:
                        print(link["href"],"Error")
                        produk_link.add(link["href"])
            


In [ ]:
getProduk("http://belimukena.com/mukena-belinda-hijau-mint/")